## NYC_Taxi Fare Prediction with Raydp

In [1]:
import ray
import os
import datetime
import pandas as pd, numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from pyspark.sql.functions import *

from raydp.spark import context
from raydp.spark.torch.estimator import TorchEstimator
from raydp.spark.utils import random_split

### Initialize ray cluster

In [ ]:
# Firstly, You should init or connect to a ray cluster. Note that you should set include_java to True
# For more config in ray, you can refer the ray doc
# ray.init(address="auto", redis_password="123")
ray.init(include_java=True)

### Create spark session based on Raydp api

In [ ]:
# After initialize ray cluster, you can use the raydp api to get a spark session
spark = context.init_spark("raydp use case", 2, 1, 5*1024*1024*1024)

### Load and process data based on spark api

In [4]:
# Then you can operate as you are using spark
data = spark.read.format("csv").option("header", "true") \
        .option("inferSchema", "true") \
        .load("/mnt/DP_disk8/nyc_taxi.csv")

In [5]:
train_data = data
# filter data
train_data_select = train_data.select("pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count","fare_amount") \
    .filter(train_data['pickup_longitude']<=-73.0) \
    .filter(train_data['pickup_longitude']>=-74.3) \
    .filter(train_data['dropoff_longitude']<=-73.0) \
    .filter(train_data['dropoff_longitude']>=-74.3) \
    .filter(train_data['pickup_latitude']<=41.7) \
    .filter(train_data['pickup_latitude']>=40.6) \
    .filter(train_data['dropoff_latitude']<=41.7) \
    .filter(train_data['dropoff_latitude']>=40.6) \
    .filter(train_data['passenger_count']<=10) \
    .filter(train_data['fare_amount'] >= 0.0) \
    .filter(abs(train_data['dropoff_longitude']-train_data['pickup_longitude'])<=5) \
    .filter(abs(train_data['dropoff_latitude']-train_data['pickup_latitude'])<=5)
# add two new features
train_data_select = train_data_select.withColumn("abs_diff_longitude", abs(col("dropoff_longitude")-col("pickup_longitude"))).withColumn("abs_diff_latitude", abs(col("dropoff_latitude") - col("pickup_latitude")))

In [ ]:
train_data_select.show()

### Model training and evaluation using Raydp with Pytorch

In [7]:
# Define the model, loss function and optimizer
class NYC_Model(nn.Module):
    def __init__(self):
        super(NYC_Model, self).__init__()
        
        self.fc1 = nn.Linear(7, 14)
        self.fc2 = nn.Linear(14, 7)
        self.fc3 = nn.Linear(7, 1)

    def forward(self, x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

nyc_model = NYC_Model()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(nyc_model.parameters(), lr=0.001)

In [8]:
# split data into train_dataset and test_dataset
train_df, test_df = random_split(train_data_select, [0.7, 0.3])
# train_df = train_data_select.sample(False, 0.0005)
# test_df = train_data_select.sample(False, 0.0001)

In [9]:
# Create a distributed estimator based on the raydp api
estimator = TorchEstimator(num_workers=4, model=nyc_model, optimizer=optimizer, loss=criterion, feature_columns=["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count","abs_diff_longitude", "abs_diff_latitude"], label_column="fare_amount", batch_size=100, num_epochs=100)

In [ ]:
# Train the model
estimator.fit(train_df)

In [ ]:
# Evaluate the model
estimator.evaluate(test_df)

In [12]:
# shutdown raydp and ray
estimator.shutdown()
context.stop_spark()
ray.shutdown()